#### 사용자정의 데이터 셋과 모델과 학습
- iris.csv ==> 사용자 정의 데이터셋
- DNN 모델 ==> 사용자 정의 모델 


[1] 모듈로딩 및 데이터 준비

In [30]:
# 모듈 로딩
import torch    # 텐서 관련 모듈
import torch.nn as nn  # 인공신경망 관련 모듈
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F  # 인공신경망 관련 함수들 모듈 (손실 함수, 활성화 함수 등)

# 성능 측정 관련 모듈
from torchmetrics.classification import F1Score  # F1 점수 관련 모듈
from torchmetrics.regression import *  # 회귀 성능 관련
from torchmetrics.classification import *  # 분류 성능 관련

# 모델 정보 출력 모듈
from torchinfo import summary  # 모델 구조 및 정보 관련 모듈

# 데이터 처리 및 시각화 모듈
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [31]:
# 데이터
DATA_FILE = r'C:\Users\zizonkjs\머신러닝,딥러닝\data\iris.csv'
irisdf=pd.read_csv(DATA_FILE)
irisdf

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [32]:
# target 컬럼 수치화
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
label.fit(irisdf['variety'])
irisdf['variety']=label.transform(irisdf['variety'])
irisdf

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


[2] 사용자 정의 데이터셋 클래스 생성 <hr>

In [33]:
# 클래스목적 : 학습용 데이터셋 Tensor화 및 전처리
# 클래스 이름 : CustomDataSet
# 부모클래스 : torch.util.data.DataSet
# 클래스 매개변수 : featuredf, targetdf


class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, featuredf, targetdf):
        super().__init__()
        self.featuredf= featuredf
        self.targetdf = targetdf
        self.n_rows = featuredf.shape[0]
        self.n_features = featuredf.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 특정 index의 데이터와 타겟 반환 메서드 -> Tensor로 반환해줘야함
    def __getitem__(self, idx):
        featureTS=torch.FloatTensor(self.featuredf.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetdf.iloc[idx].values)
        return featureTS, targetTS


[3] 데이터셋 인스턴스 생성 <hr>

In [34]:
featuredf = irisdf[irisdf.columns[:-1]]  # 마지막 열을 제외한 모든 열 (특성)
targetdf = irisdf[[irisdf.columns[-1]]]    # 마지막 열 (타겟)
irisds = CustomDataset(featuredf, targetdf)

print(irisds.n_features, irisds.n_rows)
print(irisds[0])


4 150
(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))


[4] 데이터로더 인스턴스 생성

In [35]:
# 필요한것 : Dataset  인스턴스, Batch_size=1
irisdl=DataLoader(irisds, batch_size=5)

In [36]:
for datats, targetts in irisdl:
    print(datats.shape, targetts.shape)
    break

torch.Size([5, 4]) torch.Size([5, 1])


In [37]:
# 함수기능 : 파일 확장자별 데이터프레임 변환 기능
# 함수 이름 : convertDataFrame
# 매개 변수 : data_path
# 함수결과 : DataFrame

def convertDataFrame(data_path, exit_header=0):
    ext = data_path.rsplit('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, header=exit_header )
    elif ext == 'json':
        return pd.read_json(data_path, header=exit_header)
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(data_path, header=exit_header)
    else:
        return pd.read_table(data_path, header=exit_header)
    

In [38]:
# 클래스기능 : 파일기반 데이터셋
# 클래스이름 : FileDataset
# 매개 변수 : data_path 파일경로
# 부모클래스 : utils.data.Dataset

class FileDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, data_path):
        
        super().__init__()

        datadf=convertDataFrame(data_path)
        self.featuredf=datadf[datadf.columns[:-1]]
        self.targetdf = datadf[datadf.columns[-1:]]

        self.n_features=datadf.shape[1]
        self.n_rows = self.featuredf.shape[0]


    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 특정 index의 데이터와 타겟 반환 메서드 -> Tensor로 반환해줘야함
    def __getitem__(self, idx):
        featureTS=torch.FloatTensor(self.featuredf.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetdf.iloc[idx].values)
        return featureTS, targetTS

[5] 모델준비

In [39]:

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4,10)
        self.hidden_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,3)

    # 순방향 학습 메서드
    def forward(self, x):
        y= F.relu(self.in_layer(x))
        y= F.relu(self.hidden_layer(y))
        return self.out_layer(y)


In [40]:
model = CustomModel()
print(model)
summary(model)

CustomModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Param #
CustomModel                              --
├─Linear: 1-1                            50
├─Linear: 1-2                            330
├─Linear: 1-3                            93
Total params: 473
Trainable params: 473
Non-trainable params: 0

In [44]:
for dataTS, targetTS in irisdl:
    print(dataTS.shape, targetTS.shape)

    pre_y = model(dataTS)
    print(pre_y.shape, targetTS.reshape(-1).shape)


    break

torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
